In [1]:
## Load Libraries
import os
import requests, zipfile, io

os.getcwd()

'/output'

In [2]:
# load data into platform
url = requests.get('https://he-s3.s3.amazonaws.com/media/hackathon/deep-learning-challenge-1/identify-the-objects/a0409a00-8-dataset_dp.zip')
data = zipfile.ZipFile(io.BytesIO(url.content))
data.extractall()

In [4]:
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.layers.pooling import GlobalAveragePooling2D



## load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# function to read image
def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (300, 300))
    return img

TRAIN_PATH = 'train_img/'
TEST_PATH = 'test_img/'

train_img, test_img = [],[]
for img_path in tqdm(train['image_id'].values):
    train_img.append(read_img(TRAIN_PATH + img_path + '.png'))

for img_path in tqdm(test['image_id'].values):
    test_img.append(read_img(TEST_PATH + img_path + '.png'))

# normalize images
x_train = np.array(train_img, np.float32) / 255.
x_test = np.array(test_img, np.float32) / 255.

# target variable - encoding numeric value
label_list = train['label'].tolist()
Y_train = {k:v+1 for v,k in enumerate(set(label_list))}
y_train = [Y_train[k] for k in label_list]
y_train = np.array(y_train)

y_train = to_categorical(y_train)




100%|██████████| 1732/1732 [00:36<00:00, 47.73it/s]


In [5]:
#Transfer learning with Inception V3
base_model = applications.Xception(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

## set model architechture
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x) 
predictions = Dense(y_train.shape[1], activation='softmax')(x) 
model = Model(input=base_model.input, output=predictions)

model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])

model.summary()



83501056/83683744 [============================>.] - ETA: 0s____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 300, 300, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 149, 149, 32)  128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 149, 149, 32)  0           block1_conv1_bn[0][0]            
_______________________________

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [8]:
filepath = "weights-improvement-xception-{epoch:02d}-{acc:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

batch_size = 15 # tune it
epochs = 20 # increase it

train_datagen = ImageDataGenerator(
        rotation_range=45,
        width_shift_range=0.1,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='reflect',
        vertical_flip=True,
        horizontal_flip=True)

train_datagen.fit(x_train)



In [10]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=600,
    epochs=2,
    callbacks=callbacks_list
)



Epoch 1/2
600/600 [==============================] - 721s - loss: 0.3439 - acc: 0.9028   
Epoch 2/2
600/600 [==============================] - 716s - loss: 0.1582 - acc: 0.9537   


In [11]:
predictions = model.predict(x_test, verbose=1)

predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in Y_train.items()}
pred_labels = [rev_y[k] for k in predictions]

sub = pd.DataFrame({'image_id':test.image_id, 'label':pred_labels})
sub.to_csv('sub_xception_b15_95.csv', index=False) ## ~0.59

1732/1732 [==============================] - 47s    


In [12]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=600,
    epochs=1,
    callbacks=callbacks_list
)


Epoch 1/1
600/600 [==============================] - 719s - loss: 0.0957 - acc: 0.9728   


In [15]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=600,
    epochs=1,
    callbacks=callbacks_list
)


Epoch 1/1
600/600 [==============================] - 719s - loss: 0.0639 - acc: 0.9811   


In [16]:
predictions = model.predict(x_test, verbose=1)

predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in Y_train.items()}
pred_labels = [rev_y[k] for k in predictions]

sub = pd.DataFrame({'image_id':test.image_id, 'label':pred_labels})
sub.to_csv('sub_xception_b15_98.csv', index=False) ## ~0.59

1732/1732 [==============================] - 46s    


In [17]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=600,
    epochs=1,
    callbacks=callbacks_list
)


Epoch 1/1
600/600 [==============================] - 717s - loss: 0.0759 - acc: 0.9762   


In [18]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=600,
    epochs=1,
    callbacks=callbacks_list
)


Epoch 1/1
600/600 [==============================] - 720s - loss: 0.0542 - acc: 0.9847   


In [19]:
predictions = model.predict(x_test, verbose=1)

predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in Y_train.items()}
pred_labels = [rev_y[k] for k in predictions]

sub = pd.DataFrame({'image_id':test.image_id, 'label':pred_labels})
sub.to_csv('sub_xception_b15_98_top.csv', index=False) ## ~0.59

1732/1732 [==============================] - 46s    
